# Práctica 2: Implementación de Algoritmos de Procesamiento de Imágenes

Esta práctica tiene como objetivo implementar operaciones de procesamiento de imagen utilizando métodos manuales, sin recurrir a funciones prediseñadas en librerías de tratamiento de imágenes. Al desarrollar estas implementaciones, se busca comprender tanto el funcionamiento de estos algoritmos como los efectos que tienen sobre las imágenes de entrada. 

Para esta práctica, utilizaremos Python 3.x junto con bibliotecas de procesamiento como Numpy, SciPy y Scikit-learn. El trabajo se centra en el suavizado de imágenes mediante filtros gaussianos en el dominio espacial y en el dominio de frecuencia. Cada una de las operaciones implementadas se detallará a continuación.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.data as data
img = data.camera()


## Filtrado Espacial - Suavizado Gaussiano 1D

La función `gaussianFilterSpatial1D(sigma)` genera un kernel gaussiano unidimensional con un parámetro de suavizado \( \sigma \). Este kernel se utiliza para aplicar un filtro de suavizado en el dominio espacial de una imagen.

**Parámetros:**
- `sigma`: Controla el ancho del filtro y la cantidad de suavizado. A mayor valor de \( \sigma \), el filtro será más amplio y el efecto de suavizado será mayor.

**Detalles de implementación:**
- El tamaño del kernel, \( N \), se calcula como \( N = 2 \lceil 3\sigma \rceil + 1 \), para asegurar que el kernel tenga un tamaño suficiente para capturar la forma de la función gaussiana.
- La posición central de la gaussiana, \( x = 0 \), se sitúa en el punto \( \lfloor N/2 \rfloor + 1 \).

Este filtro es el primer paso en la implementación del suavizado Gaussiano bidimensional.


In [ ]:
def gaussianFilterSpatial1D(sigma):
    N = int(2 * np.ceil(3 * sigma) + 1)
    center = (N // 2) + 1
    kernel = np.zeros(N)
    # Rellenar el kernel con la formula
    for x in range(N):
        kernel[x] = np.exp(-((x - center)**2) / (2 * sigma ** 2))
    # Normaliza el kernel para que la suma sea 1
    kernel /= np.sum(kernel)
    
    return kernel

In [ ]:
sigmas = [15, 20, 30, 40, 50, 60]  # Lista de diferentes valores de sigma
plt.figure(figsize=(10, 6))

# Generamos y graficamos el perfil Gaussiano para cada sigma
for sigma in sigmas:
    kernel = gaussianFilterSpatial1D(sigma)
    center_index = len(kernel) // 2  # Índice central del kernel
    x_values = np.arange(-center_index, center_index + 1)  # Crear eje x centrado
    plt.plot(x_values, kernel, label=f"sigma={sigma}")

# Configuramos los detalles de la gráfica
plt.title("Gaussian Kernel Profiles for Different Sigma Values (Centered)")
plt.xlabel("Index (Centered)")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()


## Filtrado Espacial - Suavizado Gaussiano 2D

La función `ApplyGaussianFilterSpatial(inImage, sigma)` aplica un filtro Gaussiano bidimensional sobre la imagen de entrada `inImage` usando un filtro de tamaño \( N \times N \) con un parámetro \( \sigma \).

**Parámetros:**
- `inImage`: Imagen de entrada sobre la cual se aplicará el suavizado.
- `sigma`: Controla el grado de suavizado, influenciando el tamaño del kernel.

**Detalles de implementación:**
- La función implementa el filtro Gaussiano bidimensional de forma eficiente, aplicando primero un kernel unidimensional 1xN y luego su transpuesto Nx1. Esto aprovecha la separabilidad del filtro Gaussiano, reduciendo la complejidad de la operación.
- El resultado es una imagen suavizada en la que se han atenuado los detalles y el ruido de alta frecuencia.

Este suavizado en el dominio espacial es útil para reducir ruido en la imagen sin pérdida significativa de información.

In [ ]:
def ApplyGaussianFilterSpatial(inImage, sigma):
    conv = gaussianFilterSpatial1D(sigma)
    # Usar apply_along_axis  para aplicar a las filas la funcion
    img = np.apply_along_axis(lambda x: np.convolve(x, conv, mode="same"), axis=1, arr=inImage)
    # Aplicar otra vez para las columnas
    filter = np.apply_along_axis(lambda x: np.convolve(x, conv, mode="same"), axis=0, arr=img) 
    return filter, (conv, img)

In [ ]:
sigma = 15
filtered_image, (conv, intermediate_image) = ApplyGaussianFilterSpatial(img, sigma)

# Creamos los subplots para visualizar cada paso
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

# Imagen original
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Imagen Original')
axes[0].axis('off')

# Resultado de la convolución en las filas
axes[1].imshow(intermediate_image, cmap='gray')
axes[1].set_title('Convolución en Filas')
axes[1].axis('off')

# Imagen final filtrada (después de la convolución en columnas)
axes[2].imshow(filtered_image, cmap='gray')
axes[2].set_title('Imagen Filtrada')
axes[2].axis('off')

# Graficamos el kernel como una línea
axes[3].plot(conv)
axes[3].set_title('Filtro Gaussiano (Kernel 1D)')
axes[3].set_xlabel("Index")
axes[3].set_ylabel("Value")

plt.show()


## Filtrado en Frecuencia - Generación del Filtro Gaussiano

La función `gaussianFilterFrec(inImage, sigma)` calcula un filtro gaussiano en el dominio de frecuencia con dimensiones iguales a las de la imagen de entrada `inImage` y un parámetro de suavizado \( \sigma \).

**Parámetros:**
- `inImage`: Imagen de entrada utilizada para definir el tamaño del filtro.
- `sigma`: Controla el ancho de la gaussiana y la cantidad de suavizado en el dominio de frecuencia.

**Detalles de implementación:**
- Este filtro es una representación Gaussiana en el dominio de frecuencia y se aplica sobre una imagen transformada a este dominio.
- La ventaja de aplicar filtros en el dominio de frecuencia es que permite realizar convoluciones de forma más eficiente usando la Transformada de Fourier.

Este filtro en el dominio de frecuencia se utilizará en la siguiente función para aplicar suavizado Gaussiano en dicho dominio.


In [ ]:
def gaussianFilterFrec(inImage, sigma):
    # Obtenemos las dimensiones
    N, M = inImage.shape
    # Obteneter la transformada de fourier
    dft = np.fft.fft2(inImage)
    img_frec = np.fft.fftshift(dft)
    
    # Crea un filtro gaussiano en el dominio de la frecuencia
    x = np.linspace(-M // 2, M // 2, M)
    y = np.linspace(-N // 2, N // 2, N)
    X, Y = np.meshgrid(x, y)
    
    # Filtro gaussiano
    gaussian_kernel = np.exp(-(X**2 + Y**2) / (2 * sigma**2))
    
    # NECESARIO VOLVER A ESPACIAL ?
    
    # Aplicar el filtro
    filtered_image_frec = img_frec * gaussian_kernel
    
    # Transformada inversa para volver al dominio espacial
    filtered_image = np.fft.ifftshift(filtered_image_frec)
    filtered_image = np.fft.ifft2(filtered_image)
    return np.abs(filtered_image), (np.abs(dft), np.abs(img_frec), np.abs(filtered_image_frec), gaussian_kernel)

In [ ]:
sigmas = [10, 20, 30, 40, 50, 60, 80]  # Lista de diferentes valores de sigma
plt.figure(figsize=(10, 6))

# Generamos y graficamos el perfil Gaussiano para cada sigma
for sigma in sigmas:
    _, (_, _, _, kernel) = gaussianFilterFrec(img, sigma)
    center_row = kernel[kernel.shape[0] // 2, :]  # Perfil central del kernel
    plt.plot(center_row, label=f"sigma={sigma}")

# Configuramos los detalles de la gráfica
plt.title("Gaussian Kernel Profiles for Different Sigma Values")
plt.xlabel("Index")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
def ApplyGaussianFilterFrec(inImage, sigma):
    filtered_image_frec, _ = gaussianFilterFrec(inImage, sigma)
    return filtered_image_frec

## Filtrado en Frecuencia - Aplicación del Filtro Gaussiano

La función `ApplyGaussianFilterFrec(inImage, sigma)` aplica el filtro Gaussiano previamente generado sobre la imagen de entrada `inImage` en el dominio de frecuencia.

**Parámetros:**
- `inImage`: Imagen de entrada que será suavizada.
- `sigma`: Controla la intensidad del suavizado, al definir el ancho de la gaussiana en el dominio de frecuencia.

**Detalles de implementación:**
- El filtro Gaussiano se ajusta al tamaño de la imagen de entrada y se aplica en el dominio de frecuencia.
- Al transformar la imagen al dominio de frecuencia y aplicar el filtro Gaussiano, obtenemos un suavizado que atenúa las altas frecuencias y reduce el ruido en la imagen de forma eficiente.

El suavizado en el dominio de frecuencia es particularmente útil en aplicaciones donde se busca procesar la imagen de manera eficiente y uniforme.


In [ ]:
sigma = 15
filtered_image = ApplyGaussianFilterFrec(img, sigma)

# Llamamos a la función y obtenemos las imágenes intermedias
filtered_image, (dft, img_frec, filtered_image_frec, gaussian_kernel) = gaussianFilterFrec(img, sigma)

# Creamos los subplots para visualizar cada paso
fig, axes = plt.subplots(1, 6, figsize=(20, 5))

# Imagen Original
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Imagen Original')
axes[0].axis('off')

# DFT de la Imagen
axes[1].imshow(np.log(1 + dft), cmap='gray')
axes[1].set_title('DFT de la Imagen')
axes[1].axis('off')

# DFT Centrada
axes[2].imshow(np.log(1 + img_frec), cmap='gray')
axes[2].set_title('DFT Centrada')
axes[2].axis('off')

# Filtro Gaussiano en Dominio de Frecuencia
axes[3].imshow(gaussian_kernel, cmap='gray')
axes[3].set_title('Filtro Gaussiano')
axes[3].axis('off')

# DFT Filtrada
axes[4].imshow(np.log(1 + filtered_image_frec), cmap='gray')
axes[4].set_title('DFT Filtrada')
axes[4].axis('off')

# Imagen Final Filtrada en el Dominio Espacial
axes[5].imshow(filtered_image, cmap='gray')
axes[5].set_title('Imagen Filtrada')
axes[5].axis('off')

plt.show()

# Visualización y Análisis de Resultados

Una vez aplicados los filtros Gaussiano en el dominio espacial y en el dominio de frecuencia, es importante observar los resultados para entender el efecto de cada filtro.

1. **Filtrado Espacial**: Debería observarse una reducción de ruido y suavizado de detalles finos en la imagen, manteniendo una buena percepción general de los bordes.

2. **Filtrado en Frecuencia**: Debería observarse un resultado similar en cuanto a la suavización, pero con una eficiencia computacional diferente. Al comparar ambas técnicas, podemos observar si los efectos son equivalentes o si existen diferencias perceptibles en ciertos detalles de la imagen.

Estas visualizaciones nos permiten verificar que el suavizado Gaussiano ha sido correctamente implementado y evaluar su impacto en la calidad de la imagen.


La diferencia en los valores de \(\sigma\) entre los filtros Gaussiano 1D y 2D se debe a cómo se propaga el efecto del desenfoque en cada caso y a cómo se interpretan los valores de \(\sigma\) en cada dimensión.

Aquí están los motivos principales:

### 1. **Escala de Desenfoque en 1D vs. 2D**
   - En un **filtro Gaussiano 1D**, \(\sigma\) determina la "anchura" del desenfoque en una sola dirección (horizontal o vertical). Para lograr un desenfoque perceptible, necesitas un valor de \(\sigma\) relativamente alto, ya que el filtro solo afecta una dimensión.
   - En un **filtro Gaussiano 2D**, \(\sigma\) determina la propagación en todas las direcciones, de manera que incluso un valor pequeño de \(\sigma\) produce un desenfoque significativo, ya que se aplica en ambas dimensiones (horizontal y vertical). Esto hace que la misma cantidad de desenfoque sea más perceptible en 2D que en 1D para el mismo valor de \(\sigma\).

### 2. **Acumulación del Efecto en 2D**
   - Al aplicar un filtro Gaussiano 1D en ambas direcciones (horizontal y vertical) de forma separada, el desenfoque final en 2D es el producto de los efectos de cada filtro. Esto significa que si aplicas un filtro 1D horizontal con \(\sigma = 5\) y luego un filtro 1D vertical también con \(\sigma = 5\), el desenfoque total se acumula, creando un efecto mayor en 2D.
   - En contraste, un filtro Gaussiano 2D directamente aplicado considera \(\sigma\) en ambas direcciones al mismo tiempo, y el efecto del desenfoque es menos intenso para el mismo valor de \(\sigma\) en comparación con aplicar dos veces el filtro 1D.

### 3. **Interpretación de \(\sigma\) en 1D vs. 2D**
   - En 1D, \(\sigma\) representa la desviación estándar de la distribución Gaussiana a lo largo de una sola línea. Para un desenfoque perceptible en una sola dimensión, necesitas una mayor "anchura" de la campana Gaussiana, es decir, un \(\sigma\) mayor.
   - En 2D, \(\sigma\) representa la desviación estándar en un plano bidimensional. Como el desenfoque se distribuye en ambas direcciones, un valor de \(\sigma\) pequeño ya es suficiente para abarcar un área amplia de la imagen.

### Resumen
En resumen, el filtro Gaussiano 2D distribuye el desenfoque en ambas direcciones simultáneamente, haciendo que un valor pequeño de \(\sigma\) sea suficiente para un desenfoque visible. En cambio, el filtro Gaussiano 1D afecta solo una dirección a la vez, por lo que se requiere un valor de \(\sigma\) mayor para lograr un efecto perceptible de desenfoque en una sola dimensión.


La confusión surge debido a la relación inversa de la sigma (desviación estándar) en los dos dominios:

    Filtro en el dominio espacial:
        Aplica una convolución directa con una función gaussiana.
        Una sigma grande significa una gaussiana más ancha, lo que provoca un mayor desenfoque porque se promedian más píxeles.
        Necesitas una sigma grande para ver un efecto notable de suavizado.

    Filtro en el dominio de la frecuencia:
        Multiplica la transformada de Fourier de la imagen por una función gaussiana en el dominio de la frecuencia.
        Una sigma pequeña en el dominio de la frecuencia atenúa más las altas frecuencias, resultando en un mayor desenfoque en el dominio espacial después de la transformada inversa.
        Por eso, con una sigma pequeña observas más efecto en la imagen final.

Importante: La gaussiana en el dominio de la frecuencia sigue siendo un filtro de paso bajo porque reduce las componentes de alta frecuencia.